# Math 597

## Project #3

## Damyn Chipman

---

### Introduction

The final set of equations we will look at in 1D are elliptic type equations. Elliptic equations arise in a variety of physics applications and require a slightly different approach when solving them than hyperbolic equations.

Primarily, we will need to take a good look at the boundary conditions imposed on elliptic problems as they dicate how the solution behaves. We'll look at how to apply both a Dirichlet and Neumann boundary condition to our elliptic solver.

We'll go through the derivation of the local and global matrices we will construct, as well as the code and algorithms required to generate them. We'll also do our standard convergence analysis.

As always, the code for this project is housed on GitHub on the [HydroForest](https://github.com/DamynChipman/HydroForest.git) repository.

### Problem Statement

We will consider the following boundary value problem (BVP):

$$\begin{align}
PDE: &\ \  \frac{d^2 q(x)}{dx^2} + q(x) = f(x), & \forall x \in \Omega = [-1,1] \\
BC: &\ \  q(x)|_{x=-1} = g = 0 \\
BC: &\ \  \frac{dq(x)}{dx}|_{x=1} = h = -\pi \\
\end{align}$$

where $f(x) = (1 - \pi^2) \sin(\pi x)$ and the exact solution is known to be $q(x) = \sin(\pi x)$.

This equation is known as Helmholtz equation.

### From PDE to Element Linear Systems

Let's use our standard Galerkin approach to form a system of linear equations on the reference element.

We start by expressing $q$ and $f$ in terms of an expansion of basis functions:

$$\begin{align}
q(x) \approx q_N^{(e)} = \sum_{j=0}^N \psi_j(x) q_j^{(e)} \\
f(x) \approx f_N^{(e)} = f(q_N^{(e)}) \\
\end{align}$$

The CG and DG approaches diverge from the start for this case, so let's start with the CG approach.

#### Continuous Galerkin (CG) Method

We multiply our PDE by a test function that is the same as our basis function (the Galerkin principle) and integrate over an individual element:

$$\begin{align}
\int_{\Omega_e} \psi_i(x) \frac{d^2 q^{(e)}_{N}(x)}{dx^2} d\Omega_e + \int_{\Omega_e} \psi_i(x) q^{(e)}_N (x) d\Omega_e &= \int_{\Omega_e} \psi_i(x) f^{(e)}_N d\Omega_e
\end{align}$$

Using the product rule, we expand the first term to give us:

$$\begin{align}
\int_{\Omega_e} \Big( \frac{d}{dx} \big[ \psi_i(x) \frac{dq^{(e)}_N(x)}{dx} \big] - \frac{d\psi_i(x)}{dx} \frac{dq^{(e)}_N(x)}{dx} \Big) d\Omega_e + \int_{\Omega_e} \psi_i(x) q^{(e)}_N (x) d\Omega_e &= \int_{\Omega_e} \psi_i(x) f^{(e)}_N d\Omega_e
\end{align}$$

Now using the Fundamental Theorem of Calculus, we get:

$$\begin{align}
\Big[ \psi_i(x) \frac{dq^{(e)}_N(x)}{dx} \Big]_{\Gamma_e} - \int_{\Omega_e} \frac{d\psi_i(x)}{dx} \frac{dq^{(e)}_N(x)}{dx} d\Omega_e + \int_{\Omega_e} \psi_i(x) q^{(e)}_N (x) d\Omega_e &= \int_{\Omega_e} \psi_i(x) f^{(e)}_N d\Omega_e
\end{align}$$

Now plug in our approximation and expansion:

$$\begin{align}
\Big[ \psi_i(x) \frac{dq^{(e)}_N(x)}{dx} \Big]_{\Gamma_e} - \sum_{j=0}^{N} \int_{\Omega_e} \frac{d\psi_i(x)}{dx} \frac{d\psi_j(x)}{dx} d\Omega_e q^{(e)}_j + \sum_{j=0}^{N} \int_{\Omega_e} \psi_i(x) \psi_j(x) d\Omega_e q^{(e)}_j &= \sum_{j=0}^{N} \int_{\Omega_e} \psi_i(x) \psi_j(x) d\Omega_e f^{(e)}_j
\end{align}$$

Next we transform to the reference element:

$$\begin{align}
\Big[ \psi_i(x) \frac{dq^{(e)}_N(x)}{dx} \Big]_{\Gamma_e} - \sum_{j=0}^{N} \int_{\hat{\Omega}} \frac{d\psi_i(\xi)}{d\xi} \frac{d\psi_j(\xi)}{d\xi} \frac{d\xi}{dx} d\hat{\Omega} q^{(e)}_j + \sum_{j=0}^{N} \int_{\hat{\Omega}} \psi_i(\xi) \psi_j(\xi) \frac{dx}{d\xi} d\hat{\Omega} q^{(e)}_j &= \sum_{j=0}^{N} \int_{\hat{\Omega}} \psi_i(\xi) \psi_j(\xi) \frac{dx}{d\xi} d\hat{\Omega} f^{(e)}_j
\end{align}$$

Which will result in the following local element system:

$$\begin{align}
B^{(e)}_i -L^{(e)}_{ij} q^{(e)}_j + M^{(e)}_{ij} q^{(e)}_j &= M^{(e)}_{ij} f^{(e)}_j \\
\Rightarrow (-L^{(e)}_{ij} + M^{(e)}_{ij}) q^{(e)}_j &= M^{(e)}_{ij} f^{(e)}_j - B^{(e)}_i \\
\Rightarrow H^{(e)}_{ij} q^{(e)}_j &= M^{(e)}_{ij} f^{(e)}_j - B^{(e)}_i \\
\end{align}$$

#### Discontinuous Galerkin (DG) Method

For the DG approach, we will be using an auxilary variable to convert the second order ODE into a system of first order ODEs. We does this as follows:

Let

$$\begin{align}
Q(x) = \frac{dq(x)}{dx}
\end{align}$$

which gives us the following system to solve:

$$\begin{align}
\begin{cases}
\frac{dq(x)}{dx} = Q(x) \\
\frac{dQ(x)}{dx} + q(x) = f(x) \\
\end{cases}
\end{align}$$

Now use our Galerkin approach by multiply both equations by a test function and integrating over an element:

$$\begin{align}
\begin{cases}
\int_{\Omega_e} \psi_i(x) \frac{dq^{(e)}_N(x)}{dx} d\Omega_e = \int_{\Omega_e} \psi_i(x) Q^{(e)}_N(x) d\Omega_e \\
\int_{\Omega_e} \psi_i(x) \frac{dQ^{(e)}_N(x)}{dx} d\Omega_e + \int_{\Omega_e} \psi_i(x) q^{(e)}_N(x) d\Omega_e = \int_{\Omega} \psi_i(x) f^{(e)}_N(x) d\Omega_e
\end{cases}
\end{align}$$

Next, we use the product rule and the FTC to transfer a derivative off of $q$ and $Q$ onto the test functions:

$$\begin{align}
\begin{cases}
\Big[ \psi_i(x) q^{(*,e)}_N(x) \Big]\Big|_{\Gamma_e} - \int_{\Omega_e} \frac{d\psi_i(x)}{dx} q^{(e)}_N(x) d\Omega_e = \int_{\Omega_e} \psi_i(x) Q^{(e)}_N(x) d\Omega_e \\
\Big[ \psi_i(x) Q^{(*,e)}_N(x) \Big]\Big|_{\Gamma_e} - \int_{\Omega_e} \frac{d\psi_i(x)}{dx} Q^{(e)}_N(x) d\Omega_e + \int_{\Omega_e} \psi_i(x) q^{(e)}_N(x) d\Omega_e = \int_{\Omega_e} \psi_i(x) f^{(e)}_N(x) d\Omega_e
\end{cases}
\end{align}$$

Now plug in our expansion...

$$\begin{align}
\begin{cases}
\Big[ \psi_i(x) q^{(*,e)}_N(x) \Big]\Big|_{\Gamma_e} - \sum_{j=0}^{N} \int_{\Omega_e} \frac{d\psi_i(x)}{dx} \psi_j(x) d\Omega_e q^{(e)}_j = \sum_{j=0}^{N} \int_{\Omega_e} \psi_i(x) \psi_j(x) d\Omega_e Q^{(e)}_j \\
\Big[ \psi_i(x) Q^{(*,e)}_N(x) \Big]\Big|_{\Gamma_e} - \sum_{j=0}^{N} \int_{\Omega_e} \frac{d\psi_i(x)}{dx} \psi_j(x) d\Omega_e Q^{(e)}_j + \sum_{j=0}^{N} \int_{\Omega_e} \psi_i(x) \psi_j(x) d\Omega_e q^{(e)}_j = \sum_{j=0}^{N} \int_{\Omega_e} \psi_i(x) \psi_j(x) d\Omega_e f^{(e)}_j
\end{cases}
\end{align}$$

And transform to the reference element:

$$\begin{align}
\begin{cases}
\Big[ \psi_i(x) q^{(*,e)}_N(x) \Big]\Big|_{\Gamma_e} - \sum_{j=0}^{N} \int_{\hat{\Omega}} \frac{d\psi_i(\xi)}{d\xi} \psi_j(\xi) d\hat{\Omega} q^{(e)}_j = \sum_{j=0}^{N} \int_{\hat{\Omega}} \psi_i(\xi) \psi_j(\xi) \frac{dx}{d\xi} d\hat{\Omega} Q^{(e)}_j \\
\Big[ \psi_i(x) Q^{(*,e)}_N(x) \Big]\Big|_{\Gamma_e} - \sum_{j=0}^{N} \int_{\hat{\Omega}} \frac{d\psi_i(\xi)}{d\xi} \psi_j(\xi) d\hat{\Omega} Q^{(e)}_j + \sum_{j=0}^{N} \int_{\hat{\Omega}} \psi_i(\xi) \psi_j(\xi) \frac{dx}{d\xi} d\hat{\Omega} q^{(e)}_j = \sum_{j=0}^{N} \int_{\hat{\Omega}} \psi_i(\xi) \psi_j(\xi) \frac{dx}{d\xi} d\hat{\Omega} f^{(e)}_j
\end{cases}
\end{align}$$

Leading us to the following system:

$$\begin{align}
\begin{cases}
F^{(e)}_{ij} q^{(*,e)}_j - \tilde{D}^{(e)}_{ij} q^{(e)}_j = M^{(e)}_{ij} Q^{(e)}_j \\
F^{(e)}_{ij} Q^{(*,e)}_j - \tilde{D}^{(e)}_{ij} Q^{(e)}_j + M^{(e)}_{ij} q^{(e)}_j = M^{(e)}_{ij} f^{(e)}_j \\
\end{cases}
\end{align}$$

We'll look at how to solve this system as well as how to handle $F_{ij}$ and $q^{(*,e)}_j$ and $Q^{(*,e)}_j$ after we form a global system.

### From Element Linear System to Global Linear System

Just like how we used a local to global mapping matrix $ID$ in project 2, we will use the same mapping here. This also means that the direct stiffness summation (DSS) operation is fairly similar. The issue will be in how we apply the boundary conditions, but we will look at that as part of the global system.

#### CG Global System

Using the same DSS operations as before (except for the Laplacian matrix, where we use an inverse metric term instead), we get the following global system:

$$\begin{align}
(-L_{IJ} + M_{IJ}) q_J &= M_{IJ} f_J - B_I \\
\Rightarrow q_J &= (M_{IJ} - L_{IJ})^{-1} M_{IJ} f_J - (M_{IJ} - L_{IJ})^{-1} B_I
\end{align}$$

#### DG Global System

We use the same DSS operations for the DG approach as well. The flux matrix, however, is slightly different. In the hyperbolic problem in project 2, we used a global flux vector computed using the Rusanov numerical flux. Becuase our elliptic problem is steady, we do not need an upwinding scheme for the numerical flux, so we just use a centered flux:

$$\begin{align}
f^{(*,e,k)} = \frac{1}{2} (q^{(e)} + q^{(k)})
\end{align}$$

where $e$ is the element and $k$ is the element face. So when we DSS the DG element system, we will use an approach that seperates the flux matrix into a boundary vector and a flux matrix times just the state variables $q$ and $Q$. We'll provide details in the section below on boundary conditions.

Using the DSS operation, we form the following global system and solve for $Q_J$:

$$\begin{align}
\begin{cases}
B^{(q)}_I + F^*_{IJ} q_{J} - \tilde{D}_{IJ} q_J = M_{IJ} Q_J \\
B^{(Q)}_I + F^*_{IJ} Q_{J} - \tilde{D}_{IJ} Q_J + M_{IJ} = M_{IJ} f_J \\
\end{cases}
\end{align}$$

$$\begin{align}
\Rightarrow Q_J &= (M_{IJ})^{-1} (B^{(q)}_I + \hat{D}^{(q)}_{IJ} q_J)
\end{align}$$

where $\hat{D}_{IJ} = F^*_{IJ} - \tilde{D}_{IJ}$.

Now plug this into the other equation and solve for $q_J$:

$$\begin{align}
B^{(Q)}_I + \hat{D}^{(Q)}_{IJ}\big( (M_{IJ})^{-1} (B^{(q)}_I + \hat{D}^{(q)}_{IJ} q_J) \big) + M_{IJ} q_J = M_{IJ} f_J \\
\Rightarrow H_{IJ} q_J = M_{IJ} f_J - B'_I
\end{align}$$

where

$$\begin{align}
H_{IJ} &= \hat{D}^{(Q)}_{IK} M^{-1}_{KL} \hat{D}^{(q)}_{LJ} + M_{IJ} \\
B'_I &= B^{(Q)}_I - \hat{D}^{(Q)}_{IK} M^{-1}_{KJ} B^{(q)}_J
\end{align}$$

which does look quite eloquent compared to the actual ODE doesn't it?

### Boundary Conditions

We need to look at how to apply the Dirichlet boundary condition at the left edge and the Neumann boundary condition at the right edge. These will be different for both CG and DG.

#### CG Boundary Conditions

Let's consider the Neumann BC first, as those are simplier in FE methods. Consider the form of the boundary integral:

$$\begin{align}
B^{(e)}_i = \Big[ \psi_i(x) \frac{dq^{(e)}_N(x)}{dx} \Big]_{\Gamma_e}
\end{align}$$

This only lives on the boundary of the element, and disappears between elements. Also, at element boundary, all basis functions are either zero or unity, so we only need to worry about the $\frac{dq}{dx}$ piece. Fortunately, we know $\frac{dq}{dx}$ at the Neumann boundary, we simply need to create a vector with zeros everywhere except the global index point corresponding to the Neumann boundary (in this case, it's the last point). So this looks like:

$$\begin{align}
B_I = \begin{bmatrix}
... & 0 & h \\
\end{bmatrix}^T = \begin{bmatrix}
... & 0 & -\pi \\
\end{bmatrix}^T
\end{align}$$

The Dirichlet BC cannot be inforced through the boundary vector, so we will enforce it in the strong sense. This means we will manipulate the global system to force the value of $q$ at the Dirichelt boundary to be the value we need.

The global system looks like:

$$\begin{align}
(-L_{IJ} + M_{IJ}) q_J &= M_{IJ} f_J - B_I = R_I
\end{align}$$

So we change the first row of $-L_{IJ} + M_{IJ}$ to be zeros except for a one in the first row, first column. And we force the first entry in the RHS vector $R_I$ to be the value of the function at the edge. This looks like for the first row/equation of the global system:

$$\begin{align}
\begin{bmatrix}
1 & 0 & 0 & ...
\end{bmatrix}
\begin{bmatrix}
q_0 \\ q_1 \\ q_2 \\ \vdots
\end{bmatrix} = 
\begin{bmatrix}
g \\ R_1 \\ R_2 \\ \vdots
\end{bmatrix} =
\begin{bmatrix}
0 \\ R_1 \\ R_2 \\ \vdots
\end{bmatrix}
\end{align}$$

This forces the first equation in the global system to read $q_0 = g = 0$.

#### DG Boundary Conditions

Apply the DG boundary conditions are a little more tricky. We do so through the flux matrix $F_{IJ}$ and the boundary vector $B_I$.

When we go from the element linear system

$$\begin{align}
\begin{cases}
F^{(e)}_{ij} q^{(*,e)}_j - \tilde{D}^{(e)}_{ij} q^{(e)}_j = M^{(e)}_{ij} Q^{(e)}_j \\
F^{(e)}_{ij} Q^{(*,e)}_j - \tilde{D}^{(e)}_{ij} Q^{(e)}_j + M^{(e)}_{ij} q^{(e)}_j = M^{(e)}_{ij} f^{(e)}_j \\
\end{cases}
\end{align}$$

to the global linear system

$$\begin{align}
\begin{cases}
B^{(q)}_I + F^*_{IJ} q_{J} - \tilde{D}_{IJ} q_J = M_{IJ} Q_J \\
B^{(Q)}_I + F^*_{IJ} Q_{J} - \tilde{D}_{IJ} Q_J + M_{IJ} = M_{IJ} f_J \\
\end{cases}
\end{align}$$

we transfer the action of the numerical flux onto the flux vector itself. This allows us to operate with the state vectors $q_J$ and $Q_J$ freely. Forming the flux matrix then requires a new approach. We motivate it by expressing the global system first as the flux matrix from previous projects (i.e., just [-1, 0, ..., 0, 1] on the diagonal per element) times the numerical flux (centered flux in this case):

$$\begin{align}
F_{IJ} q^{(*,e,k)}_J =
\begin{bmatrix}
-1 & & & & & & & & & & & & & & \\
& \ddots & & & & & & & & & & & & & \\
& & 1 & & & & & & & & & & & & \\
& & & -1 & & & & & & & & & & & \\
& & & & \ddots & & & & & & & & & & \\
& & & & & 1 & & & & & & & & & \\
& & & & & & \ddots & & & & & & & & \\
& & & & & & & \ddots & & & & & & & \\
& & & & & & & & \ddots & & & & & & \\
& & & & & & & & & -1 & & & & & \\
& & & & & & & & & & \ddots & & & & \\
& & & & & & & & & & & 1 & & & \\
& & & & & & & & & & & & -1 & & \\
& & & & & & & & & & & & & \ddots & \\
& & & & & & & & & & & & & & 1 \\
\end{bmatrix}
\begin{bmatrix}
q^{(*,0,-1)} \\
\vdots \\
q^{(*,0,1)} \\
q^{(*,1,0)} \\
\vdots \\
q^{(*,1,2)} \\
\vdots \\
\vdots \\
\vdots \\
q^{(*,N_{e}-2,N_{e}-3)} \\
\vdots \\
q^{(*,N_{e}-2,N_{e}-1)} \\
q^{(*,N_{e}-1,N_{e}-2)} \\
\vdots \\
q^{(*,N_{e}-1,N_{e})} \\
\end{bmatrix}
\end{align}$$

Now for each element interface, we consider the numerical flux scheme we are using and write the system in terms of the state variables instead of numerical flux variables. For example, at the right edge of the first element, we would get:

$$\begin{align}
q^{(*,0,1)} = \frac{1}{2} (q_N + q_{N+1})
\end{align}$$

Rewriting the system as such yields:

$$\begin{align}
F^*_{IJ} q_J =
\begin{bmatrix}
? & & & & & & & & & & & & & & \\
& \ddots & & & & & & & & & & & & & \\
& & \frac{1}{2} & \frac{1}{2} & & & & & & & & & & & \\
& & -\frac{1}{2} & -\frac{1}{2} & & & & & & & & & & & \\
& & & & \ddots & & & & & & & & & & \\
& & & & & \frac{1}{2} & \frac{1}{2} & & & & & & & & \\
& & & & & & \ddots & & & & & & & & \\
& & & & & & & \ddots & & & & & & & \\
& & & & & & & & \ddots & & & & & & \\
& & & & & & & & -\frac{1}{2} & -\frac{1}{2} & & & & & \\
& & & & & & & & & & \ddots & & & & \\
& & & & & & & & & & & \frac{1}{2} & \frac{1}{2} & & \\
& & & & & & & & & & & -\frac{1}{2} & -\frac{1}{2} & & \\
& & & & & & & & & & & & & \ddots & \\
& & & & & & & & & & & & & & ? \\
\end{bmatrix}
\begin{bmatrix}
q_0 \\
\vdots \\
q_{N} \\
q_{N+1} \\
\vdots \\
q_{2N} \\
\vdots \\
\vdots \\
\vdots \\
q_{N_e(N-1)} \\
\vdots \\
q_{N_e(N)} \\
q_{N_e(N)+1} \\
\vdots \\
q_{N_e(N+1)-1} \\
\end{bmatrix}
\end{align}$$

Now for the boundaries. We can form this system for both $q$ and $Q$. At the left edge, we need to apply Dirichlet BC, so the numerical flux looks like:

$$\begin{align}
q^{(*,0,-1)} = \frac{1}{2} (q_{-1} + q_0) = \frac{1}{2} (g + q_0)
\end{align}$$

And for the right edge, we apply Neumann BC, which look like Dirichlet BC for $Q$:

$$\begin{align}
Q^{(*,N_e-1,N_e)} = \frac{1}{2} (Q_{N_e(N+1)-1} + Q_{N_e(N+1)}) = \frac{1}{2} (Q_{N_e(N+1)-1} + h)
\end{align}$$

Now, the last piece is to figure out what to do for $q$ on the right edge (where there is no BC enforced) and $Q$ on the left edge (again, where no BC is enforced). We do this by enforcing the natural BC for $q$ or $Q$ at their respective edges:

(Left edge for $Q$):

$$\begin{align}
Q^{(*,0,-1)} = \frac{1}{2} (Q_{-1} + Q_0) = \frac{1}{2} (Q_0 + Q_0) = Q_0
\end{align}$$

(Right edge for $q$):

$$\begin{align}
q^{(*,N_e-1,N_e)} = \frac{1}{2} (q_{N_e(N+1)-1} + q_{N_e(N+1)}) = \frac{1}{2} (q_{N_e(N+1)-1} + q_{N_e(N+1)-1}) = q_{N_e(N+1)-1} 
\end{align}$$

This results in the follwing two terms: 1) a numerical flux matrix times the state vector and 2) the boundary vector.

$$\begin{align}
F^*_{IJ} q_J + B^{(q)}_I =
\begin{bmatrix}
-\frac{1}{2} & & & & & & & & & & & & & & \\
& \ddots & & & & & & & & & & & & & \\
& & \frac{1}{2} & \frac{1}{2} & & & & & & & & & & & \\
& & -\frac{1}{2} & -\frac{1}{2} & & & & & & & & & & & \\
& & & & \ddots & & & & & & & & & & \\
& & & & & \frac{1}{2} & \frac{1}{2} & & & & & & & & \\
& & & & & & \ddots & & & & & & & & \\
& & & & & & & \ddots & & & & & & & \\
& & & & & & & & \ddots & & & & & & \\
& & & & & & & & -\frac{1}{2} & -\frac{1}{2} & & & & & \\
& & & & & & & & & & \ddots & & & & \\
& & & & & & & & & & & \frac{1}{2} & \frac{1}{2} & & \\
& & & & & & & & & & & -\frac{1}{2} & -\frac{1}{2} & & \\
& & & & & & & & & & & & & \ddots & \\
& & & & & & & & & & & & & & 1 \\
\end{bmatrix}
\begin{bmatrix}
q_0 \\
\vdots \\
q_{N} \\
q_{N+1} \\
\vdots \\
q_{2N} \\
\vdots \\
\vdots \\
\vdots \\
q_{N_e(N-1)} \\
\vdots \\
q_{N_e(N)} \\
q_{N_e(N)+1} \\
\vdots \\
q_{N_e(N+1)-1} \\
\end{bmatrix} +
\begin{bmatrix}
-\frac{1}{2}g \\
0 \\
0 \\
0 \\
0 \\
0 \\
0 \\
0 \\
0 \\
0 \\
0 \\
0 \\
0 \\
0 \\
0 \\
\end{bmatrix}
\end{align}$$

$$\begin{align}
F^*_{IJ} Q_J + B^{(Q)}_I =
\begin{bmatrix}
-1 & & & & & & & & & & & & & & \\
& \ddots & & & & & & & & & & & & & \\
& & \frac{1}{2} & \frac{1}{2} & & & & & & & & & & & \\
& & -\frac{1}{2} & -\frac{1}{2} & & & & & & & & & & & \\
& & & & \ddots & & & & & & & & & & \\
& & & & & \frac{1}{2} & \frac{1}{2} & & & & & & & & \\
& & & & & & \ddots & & & & & & & & \\
& & & & & & & \ddots & & & & & & & \\
& & & & & & & & \ddots & & & & & & \\
& & & & & & & & -\frac{1}{2} & -\frac{1}{2} & & & & & \\
& & & & & & & & & & \ddots & & & & \\
& & & & & & & & & & & \frac{1}{2} & \frac{1}{2} & & \\
& & & & & & & & & & & -\frac{1}{2} & -\frac{1}{2} & & \\
& & & & & & & & & & & & & \ddots & \\
& & & & & & & & & & & & & & \frac{1}{2} \\
\end{bmatrix}
\begin{bmatrix}
q_0 \\
\vdots \\
q_{N} \\
q_{N+1} \\
\vdots \\
q_{2N} \\
\vdots \\
\vdots \\
\vdots \\
q_{N_e(N-1)} \\
\vdots \\
q_{N_e(N)} \\
q_{N_e(N)+1} \\
\vdots \\
q_{N_e(N+1)-1} \\
\end{bmatrix} +
\begin{bmatrix}
0 \\
0 \\
0 \\
0 \\
0 \\
0 \\
0 \\
0 \\
0 \\
0 \\
0 \\
0 \\
0 \\
0 \\
\frac{1}{2}h \\
\end{bmatrix}
\end{align}$$

The algorithm for forming $F^*_{IJ}$ follows Algorithm 6.4 from the book and the code is provided below:

```C++
template<typename NumericalType>
class DGGlobalCenteredFluxMatrix : public Matrix<NumericalType> {

public:

    DGGlobalCenteredFluxMatrix(std::vector<Element1D<NumericalType>>& elements, Matrix<int>& IDMatrix, BoundaryConditionType leftBoundaryType, BoundaryConditionType rightBoundaryType) :
        Matrix<NumericalType>(elements.size()*(IDMatrix.nRows()), elements.size()*(IDMatrix.nRows()), 0) {

        //
        int N = IDMatrix.nRows()-1;
        for (auto e = 0; e < elements.size(); e++) {
            int L, R, I, J;

            // Identify left and right elements
            L = e-1;
            R = e+1;
            if (e == 0) {
                if (leftBoundaryType == BoundaryConditionType::Periodic) L = elements.size()-1;
                else L = -1;
            }
            else if (e == elements.size()-1) {
                if (rightBoundaryType == BoundaryConditionType::Periodic) R = 0;
                else R = -1;
            }

            if (L != -1) {
                I = IDMatrix(0, e);
                J = IDMatrix(N, L);
                this->operator()(I,I) = -0.5;
                this->operator()(I,J) = -0.5;
            }
            else {
                I = 0;
                this->operator()(I,I) = 1.0;
            }

            if (R != -1) {
                I = IDMatrix(N, e);
                J = IDMatrix(0, R);
                this->operator()(I,I) = 0.5;
                this->operator()(I,J) = 0.5;
            }
            else {
                I = this->nRows()-1;
                this->operator()(I,I) = 1.0;
            }
        }

    }

};
```

With the matrices formed above, we can now solve for $q_J$ using the equation we derived as part of the global linear system.

### Results

#### Plots

Below are plots of the numerical and exact solution for various orders and number of elements:

##### Exact Integration

CG | DG
- | -
![](plot_solution_CG_exact_p1_n4.png) | ![](plot_solution_DG_exact_p1_n4.png)
![](plot_solution_CG_exact_p1_n16.png) | ![](plot_solution_DG_exact_p1_n16.png)
![](plot_solution_CG_exact_p1_n64.png) | ![](plot_solution_DG_exact_p1_n64.png)
![](plot_solution_CG_exact_p16_n4.png) | ![](plot_solution_DG_exact_p16_n4.png)
![](plot_solution_CG_exact_p16_n16.png) | ![](plot_solution_DG_exact_p16_n16.png)
![](plot_solution_CG_exact_p16_n64.png) | ![](plot_solution_DG_exact_p16_n64.png)

##### Inexact Integration

CG | DG
- | -
![](plot_solution_CG_inexact_p1_n4.png) | ![](plot_solution_DG_inexact_p1_n4.png)
![](plot_solution_CG_inexact_p1_n16.png) | ![](plot_solution_DG_inexact_p1_n16.png)
![](plot_solution_CG_inexact_p1_n64.png) | ![](plot_solution_DG_inexact_p1_n64.png)
![](plot_solution_CG_inexact_p16_n4.png) | ![](plot_solution_DG_inexact_p16_n4.png)
![](plot_solution_CG_inexact_p16_n16.png) | ![](plot_solution_DG_inexact_p16_n16.png)
![](plot_solution_CG_inexact_p16_n64.png) | ![](plot_solution_DG_inexact_p16_n64.png)

#### Convergance Study

The `main` program in `examples/math597-P3/main.cpp` runs a convergance study for the following parameter sweeps:

- Scheme : CG, DG
- Integration Method : Exact, Inexact
- Basis Order : 1, 2, 4, 8, 16, 32
- Number of Elements : 4, 8, 16, 32, 64, 128

This is done with MPI on 4 ranks with each rank taking one of the scheme/integration method combos. Running it produces plots of the number of degrees of freedom vs. the $L_2$ error compared to the exact solution, which is the initial solution advected around the domain once.

The plots are shown below:

CG | DG
- | -
![](plot_convergance_CG_exact.png) | ![](plot_convergance_DG_exact.png)
![](plot_convergance_CG_inexact.png) | ![](plot_convergance_DG_inexact.png)

### Discussion

#### CG vs DG

Looking at the convergence plots, we see that the CG cases have a slightly better rate coefficient, though both CG and DG seem to be converging at the same rate. My guess would be this has to do with the numerical flux and the discontinuity across element interfaces. If we look at a plot with lower number of elements and a lower order, the CG plot, while not correct, is closer to the solution than the DG version. And the jumps between elements only increase the error locally. This is of course corrected with higher number of elements of higher orders.

#### Exact vs Inexact Integration

In terms of error, there is little difference between CG exact vs inexact and DG exact vs inexact. Viewing the plots for the DG solutions, the discontinuities practically disappear.

In terms of speed, when I ran this, the inexact cases finished a good couple seconds ahead of the exact method, which would be useful for saving time in a transient problem.

#### Numerical Round Off Error

It seems that the round off error is worse than I would have expected. While `double` precision numbers normally have 16 digits of accuracy, we are only able to get to about 12 digits with all methods. And at higher degrees of freedom, round off error drives the error significantly up. I would be interested in looking at how to drive this down, because a high order scheme should be able to give us better numerical precision here.

#### Boundary Conditions

The boundary conditions have a direct impact on the solution. As Helmholtz equation is a BVP, the main driver is the boundary conditions. When implementing the matrices above, any slight variance to a value on the boundary significantly changed the resulting solution. Thus, accurate boundary conditions must be enforced well!

### Conclusion

In this project, we explored Helmholtz equation as a boundary value problem. We implemented a CG and DG scheme using a Laplacian matrix and a system of first order ODEs, respectively. The boundary conditions were enforced through the use of boundary vectors and flux matrices. We showed that we can get expected convergence with a wide range of number of elements and polynomial orders.

### References

Giraldo, Francis X. _An Introduction to Element-Based Galerkin Methods on Tensor-Product Bases: Analysis, Algorithms, and Applications_. Vol. 24. Springer Nature, 2020.